***<h1><b>Applied Machine Learning<b></h1>***
<h2>Αλέξανδρος - Ιωάννης Δουνάκης</h2> 
<h3>3170044</h3>




## <h3>Εργασία 1 </h5>

<h3><b>Q1</b></h6>
Obtaining the Data 



In [271]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

raw_data = pd.read_csv("2757408.csv",parse_dates=["DATE"] )
raw_data.DATE = pd.to_datetime(raw_data.DATE)

raw_data2 = pd.read_csv("athens.csv",parse_dates=["DATE"])
raw_data2.DATE = pd.to_datetime(raw_data2.DATE)

Διασχίζουμε τα dataset προκειμένου να δούμε ποια δεδομένα λείπουν.
Βλέπουμε πως οι NaN τιμές ειναι πολλές στο column TAVG , η οποία σίγουρα θα χρειαστεί προκειμένου να κάνουμε την ανάλυση

In [242]:
print("RAW_DATA NAN VALUES IN TAVG \n",raw_data.TAVG.isna().sum())
print("RAW_DATA NAN VALUES IN TMAX \n",raw_data.TMAX.isna().sum())
print("RAW_DATA NAN VALUES IN TMIN \n",raw_data.TMIN.isna().sum())
print("RAW_DATA NAN VALUES IN PRCP \n",raw_data.PRCP.isna().sum())

# (raw_data.TAVG == float(0)).sum()

RAW_DATA NAN VALUES IN TAVG 
 2311
RAW_DATA NAN VALUES IN TMAX 
 944
RAW_DATA NAN VALUES IN TMIN 
 833
RAW_DATA NAN VALUES IN PRCP 
 478


Θέλοντας να φτιάξουμε τα δεδομένα χρειάζεται να αλλάξουμε τις τιμές NaN σε μηδέν έτσι ώστε να είναι πιο διαχειρίσιμες

In [272]:
raw_data.TAVG.fillna(0,inplace=True)
raw_data.TMAX.fillna(0,inplace=True)    
raw_data.TMIN.fillna(0,inplace=True)
raw_data.PRCP.fillna(0,inplace=True)


Ωστόσο στο 2o Dataset δεν φαίνεται να λείπουν δεδομένα χρήσιμα σε εμάς.

In [243]:
print("RAW_DATA2 NAN VALUES IN TAVG \n",raw_data2.TAVG.isna().sum())
print("RAW_DATA2 NAN VALUES IN TMAX \n",raw_data2.TMAX.isna().sum())
print("RAW_DATA2 NAN VALUES IN TMIN \n",raw_data2.TMIN.isna().sum())
print("RAW_DATA2 NAN VALUES IN PRCP \n",raw_data2.PRCP.isna().sum())

# (raw_data.TAVG == float(0)).sum()

RAW_DATA2 NAN VALUES IN TAVG 
 0
RAW_DATA2 NAN VALUES IN TMAX 
 0
RAW_DATA2 NAN VALUES IN TMIN 
 0
RAW_DATA2 NAN VALUES IN PRCP 
 0


Συνεπώς , πρεπει να διαμορφώσουμε τα δεδομένα τα οποία λείπουν απο το 1ο σετ δεδομένων.<br>
Αρχικά τα δεδομένα που χρειάζονται διαμόρφωση είναι όσες τιμές του TAVG == 0 εφόσον νωρίτερα τρέξαμε την <br><code>raw_data.TAVG.fillna(0,inplace=True)</code> 
<br>
Συνεπώς τα βήματα που ακολουθούμε ειναι τα εξής:
<br>
<lu>
    1. Κατατόπιση των: <br>
        <li>A: Το σύνολο των γραμμών για τις οποίες ισχύει TAVG =0 και ΤΜΑΧ != 0 και ΤΜΙΝ != 0 </li>
        <li>Β: Το σύνολο των γραμμών για τις οποίες ισχύει TAVG =0 και ΤΜΑΧ =0 ή ΤΜΙΝ =0 </li>
        <br>
    2. Για το σύνολο Α , θέτουμε την τιμη TAVG όσο το AVG των στοιχείων TMAX και TMIN (στη περίπτωση μας το AVG ταυτίζεται με το median) <br>
    3. Για το σύνολο Β , θέτουμε την τιμη TAVG ίσο με την τιμη TMAX / TMIN ή οποία ειναι διαφορετική απο μηδέν.
</lu>
<br>
THIS WORKS
we dont take into consideration the case where (TAVG > TMAX) | (TAVG < TMIN)

In [273]:

raw_data.TAVG.mask( (raw_data.TAVG == 0) & (raw_data.TMAX != 0) & (raw_data.TMAX != 0) , (raw_data.TMAX +raw_data.TMIN) /2 ,inplace =True)
raw_data.TAVG.mask(cond = (raw_data.TAVG == 0) & ((raw_data.TMAX == 0) | (raw_data.TMIN == 0)), other = raw_data.TMAX + raw_data.TMIN, inplace = True)

Παρατηρώντας τα δεδομένα του αρχικού σετ δεδομένων , παρατηρούμε πως η μονάδα μέτρησης θερμοκρασίας δεν ειναι κελσίου , οπότε κάνουμε Farencheit to Celcius.

In [274]:
raw_data.TAVG.mask(raw_data.TAVG > 0 ,( (raw_data.TAVG - 32.0) * 5.0 / 9.0) , inplace=True)


Επίσης για την διαμόρφωση των δεδομένων χρειάζεται να ελέγξουμε την πληρότητα των τιμών στο πεδίο <code>raw_data.PRCP</code> <br>
Εξετάζοντας τα δεδομένα αυτά βλέπουμε πως το ποσοστο των τιμών NaN για τα έτη 2010 μέχρι 2019 είναι πολύ μεγάλο έτσι ώστε να μπορούμε να εξάγουμε συμπεράσματα. <br>
Συνεπώς θα χρειαστούμε να αντικαταστήσουμε τα δεδομένα που λείπουν απο το 2ο dataset που μας δίνεται , την εντολή αυτή την τρέχουμε παρακάτω. <br>


In [278]:
tocount = raw_data.PRCP.loc[(raw_data.DATE > '2010-1-1') & (raw_data.DATE < '2020-1-1') ]
print("PERCENTAGE OF NAN IN PRCP THROUGH 2010 - 2019\n",((tocount == 0).sum())/tocount.size *100)

PERCENTAGE OF NAN IN PRCP THROUGH 2010 - 2019
 86.24017155110793


Raw_data2.PRCP normalization to [0.0-1.0] ,as seen in original raw_data  

In [246]:
raw_data2.PRCP.mask( raw_data2.PRCP >0 , raw_data2.PRCP/100 , inplace=True)
#raw_data2

In [270]:
#raw_data.DATE.dtypes

raw_data2 = raw_data2[~raw_data2.DATE.isin(raw_data.DATE)]
data = pd.concat([raw_data , raw_data2])
data = data[['DATE','PRCP','TAVG','TMAX','TMIN']].copy()
data = data.set_index("DATE").sort_index()
data
#data.index = data.index.sort_index()


,PRCP,TAVG,TMAX,TMIN
DATE,,,,
1955-01-01,0.00,14.166667,65.0,50.0
1955-01-02,0.08,10.833333,58.0,45.0
1955-01-03,0.00,12.5,60.0,49.0
1955-01-04,0.00,13.055556,66.0,45.0
1955-01-05,0.00,13.055556,64.0,47.0
...,...,...,...,...
2020-12-27,0.00,16.666667,0.0,0.0
2020-12-28,0.26,15.0,65.0,53.0
2020-12-29,0.00,16.111111,68.0,54.0


In [249]:
# raw_data2.loc[(raw_data.index != raw_data2.index) & (raw_data.index > '2010-1-1' & raw_data.index < '2020-1-1')]
# result = raw_data.append(raw_data2 , ignore_index=False , sort=False)
# result.tail(50)
# data = result[["STATION" ,"NAME" ,"PRCP" ,"TAVG" ,"TMAX" ,"TMIN"]]
# data
#toUse = raw_data2.loc[raw_data.DATE.sort_values() == raw_data2.DATE.sort_values()]

#result = pd.concat([raw_data , raw_data2])
#result = pd.merge(raw_data, raw_data2 , how="left" , on="DATE")
#print(result.to_string())
# result.set_index(result.DATE ,inplace=True)
#result.sort_values(by="DATE")
# result = result.sort_index()
# result
result = result[[ 'STATION' , 'NAME' ,'DATE','PRCP','TAVG','TMAX','TMIN']].copy()
result.DATE = pd.to_datetime('DATE')
result.sort_values(result.DATE)



#result.set_index('DATE' ,inplace=True)



KeyError: "['PRCP', 'TAVG', 'TMAX', 'TMIN'] not in index"